In [28]:
import sys
import numpy as np
import netCDF4
import xarray as xr
import matplotlib.pyplot as plt
import os
from PIL import Image
import pandas as pd
from netCDF4 import Dataset
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.cluster import KMeans
from tqdm import tqdm
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
from sklearn.decomposition import PCA

In [29]:
# Function to Extract features from the images
def image_feature(path, image_size):
    direc = os.listdir(path)
    model = InceptionV3(weights='imagenet', include_top=False)
    features = [];
    img_names = [];
    for i in tqdm(direc):
        fname=path + '/' +i
        print(fname)
        img=image.load_img(fname,image_size)
        x = img_to_array(img)
        x=np.expand_dims(x,axis=0)
        x=preprocess_input(x)
        feat=model.predict(x)
        feat=feat.flatten()
        features.append(feat)
        img_names.append(i)
    return features,img_names

In [30]:
sst_path = "/content/drive/MyDrive/Work/Projects-Google-Drive/DOE Xue Zheng LLNL ECRP Project/ECRP_Data_Science/Jianwu_test/sst/"

In [31]:
# the size of the new figure is 41x41
img_features, img_names = image_feature(sst_path, (41, 41))

  0%|          | 0/10 [00:00<?, ?it/s]

/content/drive/MyDrive/Work/Projects-Google-Drive/DOE Xue Zheng LLNL ECRP Project/ECRP_Data_Science/Jianwu_test/sst//ERA5_daily_sst_2021_8.jpg


 10%|█         | 1/10 [00:01<00:16,  1.87s/it]

/content/drive/MyDrive/Work/Projects-Google-Drive/DOE Xue Zheng LLNL ECRP Project/ECRP_Data_Science/Jianwu_test/sst//ERA5_daily_sst_2021_7.jpg


 20%|██        | 2/10 [00:02<00:08,  1.01s/it]

...

 80%|████████  | 8/10 [00:03<00:00,  3.32it/s]

/content/drive/MyDrive/Work/Projects-Google-Drive/DOE Xue Zheng LLNL ECRP Project/ECRP_Data_Science/Jianwu_test/sst//ERA5_daily_sst_2021_2.jpg


 90%|█████████ | 9/10 [00:04<00:00,  3.49it/s]

/content/drive/MyDrive/Work/Projects-Google-Drive/DOE Xue Zheng LLNL ECRP Project/ECRP_Data_Science/Jianwu_test/sst//ERA5_daily_sst_2021_0.jpg


100%|██████████| 10/10 [00:04<00:00,  2.32it/s]


In [34]:
#save original features
img_features_df = pd.DataFrame(img_features)
img_features_df['file_name'] = img_names
sst_csv_path = "/content/drive/MyDrive/Work/Projects-Google-Drive/DOE Xue Zheng LLNL ECRP Project/ECRP_Data_Science/Jianwu_test/csv/sst.csv"
img_features_df.to_csv(sst_csv_path, index=False)

In [35]:
#clustering based original features
k = 3
clusters = KMeans(k, random_state = 40)
clusters.fit(img_features)
image_cluster = pd.DataFrame(img_names, columns=['image'])
image_cluster["clusterid"] = clusters.labels_
image_cluster
sst_clustering_path = "/content/drive/MyDrive/Work/Projects-Google-Drive/DOE Xue Zheng LLNL ECRP Project/ECRP_Data_Science/Jianwu_test/csv/sst_clusters.csv"
image_cluster.to_csv(sst_clustering_path, index=False)

In [36]:
#conduct PCA and save its results
img_features_df = pd.DataFrame(img_features)
print(img_feature_df.shape)
pca = PCA(3)
img_reduced_features = pca.fit_transform(img_features_df)
img_reduced_features_df = pd.DataFrame(img_reduced_features)
img_reduced_features_df['file_name'] = img_names
sst_pca_csv_path = "/content/drive/MyDrive/Work/Projects-Google-Drive/DOE Xue Zheng LLNL ECRP Project/ECRP_Data_Science/Jianwu_test/csv/sst_pca.csv"
img_reduced_features_df.to_csv(sst_pca_csv_path, index=False)


(10, 2179073)


In [37]:
#clustering based reduced features
k = 3
clusters = KMeans(k, random_state = 40)
clusters.fit(img_reduced_features)
image_cluster = pd.DataFrame(img_names, columns=['image'])
image_cluster["clusterid"] = clusters.labels_
image_cluster
sst_clustering_path = "/content/drive/MyDrive/Work/Projects-Google-Drive/DOE Xue Zheng LLNL ECRP Project/ECRP_Data_Science/Jianwu_test/csv/sst_pca_clusters.csv"
image_cluster.to_csv(sst_clustering_path, index=False)